# Prepare an NSC Data Return File for Import into DRDFS Salesforce
[Detroit Regional Dollars for Scholars](https://drdfs.org/) gets data from the National Student Clearinghouse on its alumni outcomes using the [StudentTracker service](https://www.studentclearinghouse.org/colleges/studenttracker/). This program takes a file returned from NSC as input, manipulates the data into the format needed to import it into DRDFS's student information system (Salesforce CoPilot), and outputs a file ready to be imported.

There are three types of records produced from the NSC Data:
1. A Standard Academic Semester record of a student enrolled at a particular school in a particular semester
1. A Degree record of a student earning a degree at a particular school on a particular date
1. A No Enrollment Academic Semester record that no record was found for a student by the NSC

## Input (Files Needed)
- `NSC_Return.csv` = The exact file returned by the NSC.
- `schools_to_replace.csv` = A file of some schools' NSC and IPEDS names. School names in Salesforce are sourced from IPEDS and don't always neatly match the same school's name as reported by the NSC. Thus, this file provides a convenient way to store known NSC Name to IPEDS name mappings to use to streamline merging in the schools data.

## Output
- `NSC_Import_Academic_Semesters.csv` = The Standard Academic Semesters file that is ready to be imported into Salesforce.
- `NSC_Import_Degrees.csv` = The Degrees file that is ready to be imported into Salesforce.

In [ ]:
import pandas as pd
import numpy as np
import csv
from tools import salesforce_to_dataframe, year_to_period, degree_type
from creds import sf

## Step 1
Load `NSC_Return.csv`, drop unnecessary columns, and format the ID field to prepare it to merge with the student data.

In [ ]:
# Read in the data returned from NSC
nsc = pd.read_csv('data/NSC_Return.csv')

# Get rid of empty columns
nsc = nsc.dropna(axis=1, how='all')

# Convert float to int to match the format needed by Salesforce
types = nsc.dtypes
for column in types.index:
    if types[column] == np.float64:
        nsc.loc[:,column] = nsc[column].astype('Int64')

# Strip the '_' to get the student's Salesforce ID but in all caps
nsc.loc[:,'ID'] = nsc['Requester Return Field'].str.strip('_')

## Step 2
Get the student data from the Salesforce API and merge it with the NSC data to get the Salesforce ID for the students.

In [ ]:
# Get student data from Salesforce API to map to Salesforce IDs
r = sf.query_all("SELECT Id, FirstName, LastName FROM Contact WHERE RecordTypeId = '0124T000000giN2'")
stus = salesforce_to_dataframe(r)

# Create new columns where "ID" and "LastName" are uppercase to match NSC data
stus.loc[:,'ID'] = stus['Id'].str.upper()

# Rename "Id" to "Student_id" to avoid confusion with other IDs later
stus = stus.rename(columns={'Id': 'Student_id'})

# Merge in the student data to get the properly-cased Salesforce IDs
merged_stu = nsc.merge(stus,
                       on='ID',
                       how='left')

In [ ]:
# Show any NSC records where the students weren't matched to the Salesforce data
merged_stu[merged_stu.Student_id.isnull()]

## Step 3
Manipulate the columns of the data returned from the NSC into the format needed by Salesforce, including dropping unnecessary columns, and split into "found" (Standard Academic Semester and Degree) and "not found" (No Enrollment Academic Semester) records.

In [ ]:
# Create a column to note the 'Data Source' as 'NSC'
merged_stu.loc[:,'Data Source'] = 'NSC'

# Split the records into those found by NSC and those not found
found = merged_stu[ merged_stu['Record Found Y/N'] == 'Y'].copy()
not_found = merged_stu[ merged_stu['Record Found Y/N'] == 'N'].copy()

# Convert all string date columns to datetime
found.loc[:,['Search Date',
           'Enrollment Begin',
           'Enrollment End',
           'Graduation Date']] = found[['Search Date',
                                        'Enrollment Begin',
                                        'Enrollment End',
                                        'Graduation Date']].apply(pd.to_datetime,
                                                                  format='%Y%m%d')

# Use year_to_period func to determine period based on enrollment begin date
found.loc[:,'Period'] = found['Enrollment Begin'].apply(year_to_period)

## Step 4 - Standard Academic Semesters and Degrees Only
Use `schools_to_replace.csv` to replace the names of the schools to make the merge easier.

In [ ]:
# Read in a few NSC School Name to IPEDS School Name mappings to fix names
with open('schools_to_replace.csv', 'r') as schools_to_replace:
    schools = {}
    reader = csv.reader(schools_to_replace)
    next(reader) # Skip header row
    for row in reader:
        schools[row[0].upper()] = row[1].upper() # Ensure uppercase

# Replace a few school names to better match IPEDS names
found.loc[:,'College Name'] = found['College Name'].replace(schools)

## Step 5 - Standard Academic Semesters and Degrees Only
Get the schools data from the Salesforce API and merge it with the NSC data to get the Salesforce ID for the school accounts.

In [ ]:
# Get school data from Salesforce API to map to Salesforce IDs
r = sf.query_all("SELECT Id, Name, BillingState FROM Account WHERE RecordTypeId = '0124T000000giMm'")
schools = salesforce_to_dataframe(r)

# Set name column to uppercase to match uppercase school name from NSC
schools.loc[:,'NAME'] = schools['Name'].str.upper()

# Rename "Id" to "School_id" to avoid confusion with other IDs later
schools = schools.rename(columns={'Id': 'School_id'})

# Merge found_sems with schools to get the school's Salesforce ID
merged_school = found.merge(schools[['School_id', 'NAME']],
                               left_on='College Name',
                               right_on='NAME',
                               how='left')

For any records printed below, follow these steps:
1. Find the corresponding school in Salesforce
1. Copy/paste the name from the table below into the first column of `schools_to_replace.csv` on the next empty row
1. Copy/paste the corresponding name from Salesforce into the second column
1. Save `schools_to_replace.csv` 
1. Re-run this program and ensure that the below table comes up empty

In [ ]:
# Any records where school and name are null, the school couldn't be matched
merged_school['NAME'][ merged_school.NAME.isnull()]

## Step 6 - Standard Academic Semesters and Degrees Only
Split into Academic Semesters and Degrees records and format academic semester name column.

In [ ]:
# Split into Academic Semesters and Degrees records (Period is NaN if Degree)
found_sems = merged_school[ ~merged_school.Period.isnull()].copy()
found_degrees = merged_school[ merged_school.Period.isnull()].copy()

# Set year column to the year the enrollment started, as a string
found_sems.loc[:,'Year'] = found_sems['Enrollment Begin'].apply(lambda x: str(x.year))

# Set acad semester name column to "Period Year" as "Fall 2019" for example
found_sems.loc[:,'Academic Semester Name'] = found_sems['Period'] + ' ' + found_sems['Year']

found_sems.loc[:,'Record Type'] = '0124T000000giMk'

## Step 7 - Standard Academic Semesters Only
Get the academic semesters data from the Salesforce API and merge it with the NSC data to determine which academic semesters already exist to be omitted from the import.

In [ ]:
# Get the academic semesters from the Salesforce API
r = sf.query_all("SELECT CoFo_SIS__Student__r.Id, CoFo_SIS__School__r.Id, Name, CoFo_SIS__Period__c, "
                 "CoFo_SIS__Academic_Year_Start_Date__c, Enrollment_End__c "
                 "FROM CoFo_SIS__Academic_Semester__c WHERE RecordTypeId = '0124T000000giMk'")
sems = salesforce_to_dataframe(r)

# Merge in sems to find sems that have previously been imported to omit
sems_merged = found_sems.merge(sems,
                               left_on=['Student_id',
                                        'Academic Semester Name',
                                        'School_id'],
                               right_on=['CoFo_SIS__Student__r Id',
                                         'Name',
                                         'CoFo_SIS__School__r Id'],
                               how='left')

# Pull out sems that didn't match an existing sem in the merge to import
sems_to_load = sems_merged[ sems_merged['CoFo_SIS__School__r Id'].isnull()]

## Step 8 - Standard Academic Semesters Only
Prepare the final data for import by dropping columns that are unnecessary for the import and output `NSC_Import_Academic_Semesters.csv` of Standard Academic Semesters that is ready to be imported into Salesforce.

In [ ]:
# Drop empty columns
sems_to_load = sems_to_load.dropna(axis=1, how='all')

# Rename some columns to match Salesforce DataLoader mapping
sems_to_load = sems_to_load.rename(columns={'Enrollment Begin_x': 'Enrollment Begin',
                                            'Enrollment End_x': 'Enrollment End',
                                            'Contact ID': 'Student: Contact ID',
                                            'Period_x': 'Period',
                                            'School_id': 'School: Account ID'})

# Write the import file to a CSV ready to load
sems_to_load.to_csv('data/NSC_Import_Academic_Semesters.csv',
                    index=False,
                    date_format='%Y-%m-%d',
                    columns=['Requester Return Field',
                             'Record Found Y/N',
                             'Search Date',
                             'College Code/Branch',
                             '2-year / 4-year',
                             'Public / Private',
                             'Enrollment Begin',
                             'Enrollment End',
                             'Enrollment Status',
                             'Class Level',
                             'Enrollment Major 1',
                             'Enrollment CIP 1',
                             'Enrollment Major 2',
                             'Enrollment CIP 2',
                             'Graduated?',
                             'College Sequence',
                             'Student_id',
                             'Data Source',
                             'Period',
                             'School: Account ID',
                             'Academic Semester Name',
                             'Record Type'])

## Step 9 - Degrees Only

Merge data from `degrees.csv` with the NSC data to determine which degrees already exist to be omitted from the import.

In [ ]:
# Get the degrees data from the Salesforce API
r = sf.query_all("SELECT CoFo_SIS__Student__r.Id, CoFo_SIS__School__r.Id, Name, Degree_Major_1__c "
                 "FROM CoFo_SIS__Degree__c")
degrees = salesforce_to_dataframe(r)

# Merge in degrees to find degrees that have previously been imported to omit
degrees_merged = found_degrees.merge(degrees,
                                     left_on=['Student_id',
                                              'Degree Title',
                                              'School_id',
                                              'Degree Major 1'],
                                     right_on=['CoFo_SIS__Student__r Id',
                                               'Name',
                                               'CoFo_SIS__School__r Id',
                                               'Degree_Major_1__c'],
                                     how='left')

# Pull out degrees that didn't match an existing degree in the merge to import
degrees_to_load = degrees_merged[ degrees_merged['CoFo_SIS__School__r Id'].isnull()]

## Step 10 - Degrees Only
Prepare the final data for import by dropping columns that are unnecessary for the import and output `NSC_Import_Degrees.csv` of Degrees that is ready to be imported into Salesforce.

In [ ]:
# Pull out only those columns needed for the import
degrees_to_load = degrees_to_load[['Student_id', 'School_id', 'Graduation Date',
                                   'Degree Title', 'Degree Major 1',
                                   'Degree CIP 1', 'Data Source']]

degrees_to_load = degrees_to_load.rename(columns={'Student_id': 'Student: Contact ID',
                                                  'School_id': 'School: Account ID'})

# Fill the empty Degree Title values since they are required in Salesforce
degrees_to_load.loc[:,'Degree Title'] = degrees_to_load['Degree Title'].fillna(value='UNKNOWN')

# Assign Degree Type when possible
degrees_to_load.loc[:,'Degree Type'] = degrees_to_load['Degree Title'].apply(degree_type)

# Write the import file to a CSV ready to load
degrees_to_load.to_csv('data/NSC_Import_Degrees.csv',
                       index=False,
                       date_format='%Y-%m-%d')